In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [2]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset
from collections import Counter

In [3]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
inputs = tokenizer(['Hello', 'How are you'], padding=True, truncation=True, return_tensors='tf')
inputs

{'input_ids': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[ 101, 7592,  102,    0,    0],
       [ 101, 2129, 2024, 2017,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1]], dtype=int32)>}

In [6]:
output = model(inputs)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 5, 768), dtype=float32, numpy=
array([[[-0.30609733,  0.26222885, -0.18961737, ..., -0.16511053,
          0.10138195,  0.4119023 ],
        [-0.73900914, -0.03363667,  0.3932195 , ..., -0.1818075 ,
         -0.18388602, -0.21847905],
        [ 0.5800885 ,  0.06270925, -0.2637223 , ...,  0.3963107 ,
         -0.56836957, -0.49242973],
        [-0.605158  ,  0.16286792,  0.36690637, ..., -0.08638733,
          0.13334492,  0.28940228],
        [-0.65102273, -0.01890645,  0.24940467, ..., -0.05653275,
          0.19524042,  0.21820325]],

       [[ 0.07747278,  0.04501358, -0.16765122, ..., -0.3346343 ,
          0.16396384,  0.12080589],
        [-0.02718119, -0.71175593, -0.29173535, ..., -0.12723064,
          0.6787842 , -0.5343534 ],
        [ 0.6442044 , -0.9804521 , -0.03809471, ..., -0.60107917,
          0.16867957, -0.49630743],
        [ 0.01914811, -0.7620813 ,  0.58827186, ..., -0.32224905

In [7]:
dataset = load_dataset('jquiros/suicide')

# Preprocess dataset: Drop the first column, keep second as text, and third as label
def preprocess_data(example):
    return {"text": example["text"], "label": 1 if example["class"] == "suicide" else 0}

dataset = dataset.map(preprocess_data)
dataset = dataset.remove_columns(["Unnamed: 0", "class"])  # Remove unnecessary columns

# Splitting dataset
dataset = dataset["train"].train_test_split(test_size=0.2)


Suicide_Detection.csv:   0%|          | 0.00/167M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/232074 [00:00<?, ? examples/s]

Map:   0%|          | 0/232074 [00:00<?, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 185659
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 46415
    })
})

In [9]:
print("Sample Entries from the Training Dataset:")
print(dataset['train'][:5])

Sample Entries from the Training Dataset:
{'text': ['Anyone wanna me tortured? I’m bored so yea thought I’d ask', 'Everyday I try to hang myself and keep failing.. today I almost blacked out so I’m finally making progress! :)I’m so happy knowing that my efforts are finally starting to pay off.', 'I want to die so muchLike everything is pointless and I just hate myself so much. I want to die so bad. I don’t know what’s holding me back.', 'I love Cochise. filler filler filler filler filler filler Filler filler filler filler filler filler filler filler filler filler filler filler', "Please someone talk to me I'm fucking begging.really fighting the urge to go outside and hang myself right now. I'm fucking begging "], 'label': [0, 1, 1, 0, 1]}


In [10]:
def tokenize(batch): return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

In [11]:
dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/185659 [00:00<?, ? examples/s]

Map:   0%|          | 0/46415 [00:00<?, ? examples/s]

In [12]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 185659
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 46415
    })
})

In [13]:
dataset_encoded.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])


BATCH_SIZE = 64

def order(inp):
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `dataset_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# doing the same for test set
test_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [14]:
inp, out = next(iter(train_dataset))
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 128), dtype=int64, numpy=
array([[ 101, 3509, 2100, ...,    0,    0,    0],
       [ 101, 1061, 1005, ...,    0,    0,    0],
       [ 101, 2061, 1012, ...,    0,    0,    0],
       ...,
       [ 101, 2061, 2057, ...,    0,    0,    0],
       [ 101, 2129, 2079, ...,    0,    0,    0],
       [ 101, 2419, 4597, ...,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(64, 128), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor(
[1 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0

In [15]:
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_classes": self.fc.units,  # Store number of output classes
        })
        return config

    @classmethod
    def from_config(cls, config):
        from transformers import TFBertModel  # Import BERT model
        bert_model = TFBertModel.from_pretrained("bert-base-uncased")  # Load base model again
        return cls(bert_model, config["num_classes"])


In [16]:
classifier = BERTForClassification(model, num_classes=2)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [17]:
history = classifier.fit(
    train_dataset,
    epochs=2
)

Epoch 1/2
2901/2901 [==============================] - 4538s 2s/step - loss: 0.1021 - accuracy: 0.9618
Epoch 2/2
2901/2901 [==============================] - 4491s 2s/step - loss: 0.0550 - accuracy: 0.9803


In [18]:
classifier.evaluate(test_dataset)

726/726 [==============================] - 384s 524ms/step - loss: 0.0628 - accuracy: 0.9786


[0.06283337622880936, 0.9785629510879517]

In [23]:

import tensorflow as tf

def test_suicide_detection(texts, classifier, tokenizer):
    """
    Predicts whether input texts indicate suicidal thoughts or not.

    Args:
        texts (list): List of text strings to classify.
        classifier: Trained BERT classifier model.
        tokenizer: Tokenizer used during training.

    Returns:
        None. Prints out the text and its predicted category (suicide/non-suicide).
    """
    label_mapping = {0: "non-suicide", 1: "suicide"}

    # Tokenize input text
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="tf")

    # Get predictions
    outputs = classifier(inputs)
    predictions = tf.argmax(outputs, axis=1).numpy()

    # Map predictions to labels
    predicted_labels = [label_mapping[pred] for pred in predictions]

    # Print results
    print("\nPredicted Suicide Classification:")
    for text, label in zip(texts, predicted_labels):
        print(f"Text: '{text}' => Predicted Category: {label}")

# Example texts to classify
test_texts = [
    "I feel so hopeless and lost, life is meaningless.",
    "I am really happy and excited about my upcoming vacation!",
    "Nothing seems to work out for me. I don't know what to do anymore.",
    "what a great day today is. We won the game and Im so happy",
    "I think Im doing really good these days",
    "I dont want to live anymore",
    "Nobody loves me",
    "I want to kill my self",
    "I am starting to feel great",
    "I am going to shoot my self today I cant do this anymore",
    "I went on an amazing date today, she was really nice",
    "I am going through a hard breakup and middle of that Im financially broken. I feel like the only option is to take my life",
    "I hate my life so much",
    "Today is the best day of my life because i got graduated",
    "yoo I cant do this shit anymore Im tired",
    "today game was fireee.. damn what a match"
]

# Run the test
test_suicide_detection(test_texts, classifier, tokenizer)



Predicted Suicide Classification:
Text: 'I feel so hopeless and lost, life is meaningless.' => Predicted Category: suicide
Text: 'I am really happy and excited about my upcoming vacation!' => Predicted Category: non-suicide
Text: 'Nothing seems to work out for me. I don't know what to do anymore.' => Predicted Category: suicide
Text: 'what a great day today is. We won the game and Im so happy' => Predicted Category: non-suicide
Text: 'I think Im doing really good these days' => Predicted Category: non-suicide
Text: 'I dont want to live anymore' => Predicted Category: suicide
Text: 'Nobody loves me' => Predicted Category: suicide
Text: 'I want to kill my self' => Predicted Category: suicide
Text: 'I am starting to feel great' => Predicted Category: non-suicide
Text: 'I am going to shoot my self today I cant do this anymore' => Predicted Category: suicide
Text: 'I went on an amazing date today, she was really nice' => Predicted Category: non-suicide
Text: 'I am going through a hard brea

In [20]:
import tensorflow as tf
import shutil
from google.colab import files

# Define save paths
model_save_path = "bert_suicide_model"
tokenizer_save_path = "tokenizer"

# Save the model
classifier.save(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_save_path)

# Zip the model and tokenizer for download
shutil.make_archive(model_save_path, 'zip', model_save_path)
shutil.make_archive(tokenizer_save_path, 'zip', tokenizer_save_path)

# Download model and tokenizer
files.download(model_save_path + ".zip")
files.download(tokenizer_save_path + ".zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>